In [1]:
import duckdb
import os
import glob
from datetime import datetime, timezone

In [2]:
# ---- CONFIG ----
## vigyazz a pathra
DB_PATH = "../databases/currency_rates.duckdb"
JSON_FOLDER = "../jsons/fx_jsons"
SCHEMA_NAME = "raw"
TABLE_NAME = "raw_ingested_data"
# -----------------


In [3]:

# 1) DB csatlakozás -- ha nincs meg db file akkor letrehoz egyet
con = duckdb.connect(DB_PATH)


In [4]:

# 2) Schema létrehozása, ha nem letezik
con.execute(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME}")


In [5]:
# 3) JSON fájlok listázása
json_files = glob.glob(os.path.join(JSON_FOLDER, "*.json"))
if not json_files:
    raise Exception("Nem található JSON fájl!")

# 4) Betöltési timestamp (UTC)
load_ts = datetime.now(timezone.utc).isoformat()

# 5) Dobom a tablat, h a napi refresh utan frissuljon
con.execute(f"DROP TABLE IF EXISTS {SCHEMA_NAME}.{TABLE_NAME}")

# 6) JSON beolvasása + extra oszlopok (filename + timestamp)
con.execute(f"""
    CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} AS
    SELECT 
        *,
        '{load_ts}' AS loaded_at_utc,
        filename AS file_name
    FROM read_json('{JSON_FOLDER}/*.json', auto_detect=true, filename=true)
    """)

print("Betöltés kész.")

Betöltés kész.


In [6]:
# 7) Teszt
rowcount = con.execute(f"SELECT COUNT(*) FROM {SCHEMA_NAME}.{TABLE_NAME}").fetchone()
print("Number of rows:", rowcount[0])

Number of rows: 27


In [7]:
# 8) Teszt folyt
df = con.execute("""
    SELECT *
    FROM raw.raw_ingested_data
""").df()

df.head()

,success,terms,privacy,timeframe,start_date,end_date,source,quotes,filename,loaded_at_utc,file_name
0,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,True,1999-01-01,1999-12-31,USD,"{'1999-01-01': '{""USDANG"":1.780673,""USDAUD"":1....",../jsons/fx_jsons/usd_other_fx_rates_1999.json,2025-12-01T20:13:58.464925+00:00,../jsons/fx_jsons/usd_other_fx_rates_1999.json
1,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,True,2000-01-01,2000-12-31,USD,"{'2000-01-01': '{""USDANG"":1.79,""USDAUD"":1.5329...",../jsons/fx_jsons/usd_other_fx_rates_2000.json,2025-12-01T20:13:58.464925+00:00,../jsons/fx_jsons/usd_other_fx_rates_2000.json
2,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,True,2001-01-01,2001-12-31,USD,"{'2001-01-01': '{""USDAED"":3.67266,""USDALL"":145...",../jsons/fx_jsons/usd_other_fx_rates_2001.json,2025-12-01T20:13:58.464925+00:00,../jsons/fx_jsons/usd_other_fx_rates_2001.json
3,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,True,2002-01-01,2002-12-31,USD,"{'2002-01-01': '{""USDAED"":3.67266,""USDALL"":137...",../jsons/fx_jsons/usd_other_fx_rates_2002.json,2025-12-01T20:13:58.464925+00:00,../jsons/fx_jsons/usd_other_fx_rates_2002.json
4,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,True,2003-01-01,2003-12-31,USD,"{'2003-01-01': '{""USDAED"":3.67266,""USDALL"":131...",../jsons/fx_jsons/usd_other_fx_rates_2003.json,2025-12-01T20:13:58.464925+00:00,../jsons/fx_jsons/usd_other_fx_rates_2003.json


In [8]:
# 9) Teszt folyt 2
df = con.execute("""
    SELECT
        filename,
        loaded_at_utc,
        q.key   AS quote_date,
        q.value AS quote_value
    FROM raw.raw_ingested_data,
        json_each(quotes) AS q
""").df()

df

,filename,loaded_at_utc,quote_date,quote_value
0,../jsons/fx_jsons/usd_other_fx_rates_1999.json,2025-12-01T20:13:58.464925+00:00,1999-12-31,"{""USDANG"":1.79,""USDAUD"":1.525918,""USDAWG"":1.79..."
1,../jsons/fx_jsons/usd_other_fx_rates_2000.json,2025-12-01T20:13:58.464925+00:00,2000-12-31,"{""USDAED"":3.67246,""USDALL"":145.46158,""USDANG"":..."
2,../jsons/fx_jsons/usd_other_fx_rates_2001.json,2025-12-01T20:13:58.464925+00:00,2001-12-31,"{""USDAED"":3.67246,""USDALL"":137.694315,""USDAMD""..."
3,../jsons/fx_jsons/usd_other_fx_rates_2002.json,2025-12-01T20:13:58.464925+00:00,2002-12-31,"{""USDAED"":3.67246,""USDALL"":131.093735,""USDAMD""..."
4,../jsons/fx_jsons/usd_other_fx_rates_2003.json,2025-12-01T20:13:58.464925+00:00,2003-12-31,"{""USDAED"":3.67246,""USDALL"":113.838932,""USDAMD""..."
...,...,...,...,...
9823,../jsons/fx_jsons/usd_other_fx_rates_2008.json,2025-12-01T20:13:58.464925+00:00,2008-01-01,"{""USDAED"":3.6726,""USDALL"":82.692923,""USDAMD"":3..."
9824,../jsons/fx_jsons/usd_other_fx_rates_2012.json,2025-12-01T20:13:58.464925+00:00,2012-01-01,"{""USDAED"":3.6728,""USDAFN"":45.736288,""USDALL"":1..."
9825,../jsons/fx_jsons/usd_other_fx_rates_2016.json,2025-12-01T20:13:58.464925+00:00,2016-01-01,"{""USDAED"":3.672754,""USDAFN"":68.45,""USDALL"":125..."
9826,../jsons/fx_jsons/usd_other_fx_rates_2020.json,2025-12-01T20:13:58.464925+00:00,2020-01-01,"{""USDAED"":3.673202,""USDAFN"":78.40293,""USDALL"":..."


In [9]:
# futasd a checkpointot, h a currency_rates.duckdb.wal file (Write ahead log) ami a valtoztatasokat tartalömazza eltunjön
# vegul zard be a connectiont!
con.execute("CHECKPOINT;")
con.close()